In [1]:
''' try to get all realized structures from eqg table on the field of eq. groups
In this program do throught the list of lists
select groups that get part in final structures
try to speed up'''
'''block for Ml output added'''
'''seek pairs of branches in result'''
'''work with datatable for construct all possible structures for identification'''
'''work with leave pairs in datatable'''
'''on input get groupcontent. Realise step by step approach'''
# 220318 creation of structures from table edited - do not miss the last string of thr
# добавь отсев всех структур, для которых не посчиталась верхняя матрица
# тест на данных психологов не прошла - плохо собирает структуры
# пробую добавить ограничения, чтобы сортировать симметричные решения
# добавляю ограничения к функционалу
# 220401 нужны исправления в синтез структур по таблице groupcontent
# 220404 убрал из кода часть с генерацией groupcontent
# 220427 +- отладил навигацию по таблице ветвей. Пробую прикрутить решатель
# 220523 # need to add solution for Mll case
# помни, в файле groupcontent последняя строка д.б. nan
# 220524 changes in confl t formation
#dfconfl = pd.DataFrame(columns=[i for i in range(3)],index=range(len(data_step['F'])))
#dfconfl[2] = data_step['F']
# 220527 хочу посмотреть какие ограничения делают модель неразрешимой computing IIS
import re

import numpy as np
import pandas as pd
import string
import time
import copy
# for hybrid:
from itertools import chain
#model.Params.Threads = 8
import gurobipy as gp
from gurobipy import GRB
import sys
import math
import random
from itertools import product 
#import common_funcs_irm as cfirm



In [2]:
# all functions 

def listToString(list1):
    return (''.join(map(str, list1)))

def listToString_s(list1):
    return (' '.join(map(str, list1)))

def listToString_plus(list1):
    return ('+'.join(map(str, list1)))

def listToString__(list1):
    return ('_'.join(map(str, list1)))

num2alpha = dict(zip(range(1, 27), string.ascii_lowercase))

def Bin2Pos(x):#DectoMyNum(x):
    '''принимаю one-hot значение(бинарный формат) в виде строки
    возвращаю позицию единицы
    "00001" == 0'''
    x = int(x,2)
    for i in range(10):
        if x == pow(2, i): return(i)
    return(0)

def oh2i(vec): 
    '''convert 1h (integer vector, list) to integer number. Simply return position of 1 in vector  '''
    vec = list(map(round, vec)) # for float input convert into int
    for i in range(len(vec)): 
        if vec[i] == 1: return(i)
    return(0) 

def ohvec2i(vec, dim):
    '''conver oh list into int list
    parts of i shoul be together in vec'''
    res = []
    i = 0
    while i < len(vec):
        t = []
        for j in range(dim):
            t.append(vec[i+j])
        t = oh2i(t)
        res.append(t)
        i += dim
    return res

def i2oh(i, dim):
    if i >= dim: i = dim - 1 
    elif i < 0: i = 0
    vec = [0 for a in range(dim)]
    vec[i] = 1
    return vec

def RowDimToRange(y,dim):
    '''get pow of 2 and dim - return range((dim*i+1),(dim*(i+1)), from 0'''
    for i in range(10):
        if y == pow(2, i): return( range((dim*i),(dim*(i+1))))
    return(0)

def RowDimToRange_2(y,dim):
    '''get integer and dim - return range((dim*(i-1)+1),(dim*i)), from 0'''
    for i in range(0, 10):
        if y == i: return( range((dim*i),(dim*(i+1))))
    return(0)

def selectFromDF(dframe, left, right, dim):
    '''get selection from np.array, according to dimension
    уномение y*dframe*x, y - м.б. 0, когда нужно умножение только с одним множетелем
    добавь вариант когда х - ноль
    '''
    if not(sum(left)): range_left = range(dframe.shape[0]) # y = zero, so get all rows
    else:
        #row <- DectoMyNum(y)
        leftd_r = oh2i(left) # analize were 1 position in left term
        #print("left", leftd_r)
        range_left = RowDimToRange_2(leftd_r, dim)
        #print("r_left", range_left)
  
    if not(sum(right)): range_right = range(dframe.shape[1]) # x = zero, so get all columns
    else:
        range_right = oh2i(right) # num of 1 in vec
  
    dframe = dframe[range_left,range_right]
    return(dframe.tolist())

def Fcomb(df_str, df_col, dim): 
    '''формируем столбец результата умножения строки на столбец для текстовых переменных
    считаю что размерность матрицы и ячеек матрицы - совпадают. row and col - linear
    на вход получаю 2 списка. Выдаю список'''
    Rdf = [i for i in range(dim)]
    txt = [i for i in range(dim)]
    for m in range(dim):    #перебор колонки результата
        for i in range(dim):  # перебор по строкам
            range_str = RowDimToRange_2(0,dim)
            range_col = RowDimToRange_2(i,dim) # column data is complex, so there is range and recounting in range
            txt[i] = str(df_str[range_str[i]]) + "*" + str(df_col[range_col[m]]) # диапазон определяется через i а позиция в диапазоне через m
   
        Rdf[m] = "(" +  listToString_plus(txt) + ")"
    return(Rdf)

def m_create(numofletter, dim):
    '''create array with numbered simbols, from letter'''
    m_t = np.empty((dim*dim,dim), dtype = 'U16')
    for j in range(dim):
        for i in range(dim):
            for d in range(dim):
                m_t[i*dim+d,j] = '{}{}{}_{}'.format(string.ascii_lowercase[numofletter], j, i, d)
    return m_t 

def m_create_byname(name, dim):
    '''create array with numbered simbols, from letter'''
    m_t = np.empty((dim*dim,dim), dtype = 'U16')
    for j in range(dim):
        for i in range(dim):
            for d in range(dim):
                m_t[i*dim+d,j] = '{}{}{}_{}'.format(name, j, i, d)
    return m_t 

def m_create_byname_t(name, dim):
    '''create array with numbered simbols, from letter
    in NA style'''
    m_t = np.empty((dim*dim,dim), dtype = 'U16')
    for i in range(dim):
        for j in range(dim):
            for d in range(dim):
                m_t[i*dim+d,j] = '{}{}{}_{}'.format(name, j, i, d)
    return m_t 

def l_create(dim): return [i for i in range(dim)]

def l_create_0oh(dim):
    '''return one-hot zero vector '''
    zero = [1]
    zero = zero + [0 for i in range(dim-1)]
    return zero

def ll_create_0oh(l_num, dim):
    list_l = []
    for m in range(l_num):
        list_l.append(l_create_0oh(dim))
    return list_l
    
def lm_create(m_num, dim):
    '''return list of np matrises. Contain chars in one-hot style'''
    list_m = []
    for m in range(m_num):
        list_m.append(m_create(m, dim))
    return list_m

def list_split(ll, dim):
    if len(ll)%dim: print("incorrect list splitting")
    lol = [[] for i in range(int(len(ll)/dim))]
    cnt = 0
    for i in range(len(ll)):
        if i > 0 and not i%dim:
            cnt += 1
        lol[cnt].append(ll[i])
        
    return lol
# попробуй преобразовать а массив а затем изметить форму


def shell_find(f_text_v, mark=0):
    '''return coordinates of upper layer brakets in phrase'''
    Term1_r_brct_ind, Term1_l_brct_ind, Term1_r_brct_cnt, Term1_l_brct_cnt = 0, 0, 0, 0
    Term1_l_brct_found = 0
    while mark < len(f_text_v):         
        if f_text_v[mark] == '(': 
            Term1_l_brct_cnt = Term1_l_brct_cnt + 1
            if not Term1_l_brct_found:
                Term1_l_brct_ind = mark 
                Term1_l_brct_found = 1
        if f_text_v[mark] == ')':
            if Term1_r_brct_cnt >= Term1_l_brct_cnt: return -1, -1 # like ...)*()
            Term1_r_brct_cnt = Term1_r_brct_cnt + 1
            Term1_r_brct_ind = mark    
        if (Term1_l_brct_cnt != 0) and (Term1_r_brct_cnt == Term1_l_brct_cnt): break
        mark = mark + 1
    if Term1_l_brct_cnt != Term1_r_brct_cnt: return -1, -1
    else: return Term1_l_brct_ind, Term1_r_brct_ind

def rbracket_find(f_text_v, mark=0):
    '''return coordinates of upper layer right braket in phrase'''
    Term1_r_brct_ind, Term1_l_brct_ind, Term1_r_brct_cnt, Term1_l_brct_cnt = 0, 0, 0, 0
    Term1_l_brct_found = 0
    while mark < len(f_text_v):         
        if f_text_v[mark] == '(': 
            Term1_l_brct_cnt = Term1_l_brct_cnt + 1
            if not Term1_l_brct_found:
                Term1_l_brct_ind = mark 
                Term1_l_brct_found = 1
        if f_text_v[mark] == ')':
            if Term1_r_brct_cnt >= Term1_l_brct_cnt: return -1 # like ...)*()
            Term1_r_brct_cnt = Term1_r_brct_cnt + 1
            Term1_r_brct_ind = mark    
        if (Term1_l_brct_cnt != 0) and (Term1_r_brct_cnt == Term1_l_brct_cnt): break
        mark = mark + 1
    if Term1_l_brct_cnt != Term1_r_brct_cnt: return -1
    else: return Term1_r_brct_ind

    
def get_monomials(f_text):
    '''return list of monoms (was named substr_monomial)'''
    doc = re.sub("\+", ' ', f_text) 
    doc = re.sub("\)|\(", ' ', doc)
    vals = doc.split()
    vals  = list(dict.fromkeys(vals))
    vals.sort()
    return(vals)

def get_productparts(f_text):
    '''return list of monoms (was named substr_monomial)'''
    doc = re.sub("\*", ' ', f_text) 
    doc = re.sub("\)|\(", ' ', doc)
    vals = doc.split()
    vals  = list(dict.fromkeys(vals))
    vals.sort()
    return(vals)

def getitem(text):
    '''return first expression before +*()'''
    i = 0
    while i < len(text):
        if text[i] == '+' or text[i] == '*' or text[i] == '(' or text[i] == ')':
            return text[0:i]
        i += 1
    return text
#getitem('a_0012333*b00_3')

def polynomial_simplify(f_text_v):
    '''It opens all brackets in polynomial'''
    
    # USE get item function for item_lgth
    item_lgth = getitem(f_text_v[1:len(f_text_v)])
    item_lgth = len(item_lgth) 
    
    # like a00_0. Better to detect item_lgth
    Term1_r_brct_ind, Term1_l_brct_ind, Term1_r_brct_cnt, Term1_l_brct_cnt = [0 for _ in range(4)]
    Term2_r_brct_ind, Term2_l_brct_ind, Term2_r_brct_cnt, Term2_l_brct_cnt = [0 for _ in range(4)]
    err_code, mark = 0, 0
    store_1, store_2, str_sum, str_new = ["" for _ in range(4)]
    if(not re.search('\(', f_text_v)): return(f_text_v)
  
    while(mark < len(f_text_v)): 
        # better stop condition?
        Term1_l_brct_ind, Term1_r_brct_ind = shell_find(f_text_v, mark)
        mark = Term1_r_brct_ind
        if(max(Term1_l_brct_ind, Term1_r_brct_ind) == 0):
            #break  # there are no brackets in phrase
            if(str_new == ""): return(f_text_v) # first time on this recursion lvl
            else: return(str_new) # there was result on this lvl

        if(max(Term1_l_brct_ind, Term1_r_brct_ind) == -1):
            err_code <- 1
            break
    
        if( (Term1_r_brct_ind < len(f_text_v)-1) and (f_text_v[mark+1] == '*') ):
            # ()*smth: ()*a or ()*() 
            if(f_text_v[mark+2] != "("): 
                # monomial next after bracket
                Term2_l_brct_ind = mark+2
                # USE get item function for item_lgth
                item_lgth = getitem(f_text_v[mark+2:len(f_text_v)])
                item_lgth = len(item_lgth) 
                Term2_r_brct_ind = mark+2+item_lgth
                mark = Term2_r_brct_ind
            else: 
                # another () next after first bracket
                Term2_l_brct_ind = mark+3
                # +3 because I whant to get the brackets content
                Term2_r_brct_ind = rbracket_find(f_text_v, mark+2)
                mark = Term2_r_brct_ind
            # get what in brackets, without "(" ")"
            store_1 = polynomial_simplify(f_text_v[Term1_l_brct_ind+1:Term1_r_brct_ind]) 
            store_2 = polynomial_simplify(f_text_v[Term2_l_brct_ind:Term2_r_brct_ind])
            # get valute next after bracket # even if it is ()
            store_1 = get_monomials(store_1)
            store_2 = get_monomials(store_2)
            mult = [] 
            # monomial multiplication
            for i in range(len(store_1)):
                for j in range(len(store_2)): mult.append(store_1[i] + "*" + store_2[j])
            str_sum = "+".join(mult)
            # monomial summation
            if(str_new == ""): str_new = str_sum 
            else: str_new = "+".join([str_new, str_sum])  
            # remove used part
            f_text_v = f_text_v[0:Term1_l_brct_ind] + f_text_v[Term2_r_brct_ind+1:len(f_text_v)]
            mark = max(mark - Term2_r_brct_ind, 0)
    
        elif( (Term1_l_brct_ind > 0) and (f_text_v[Term1_l_brct_ind-1] == '*') ): 
            # a*() # этот случай не входит в первое условие видимо потому что алгорим ищет скобки, для вхождения в анализ
            # encoding: Term2*(Term1)
            # get item before '('
            ii = Term1_l_brct_ind-2
            while ii > 0:
                if f_text_v[ii] == '+' or f_text_v[ii] == '*' or f_text_v[ii] == '(' or f_text_v[ii] == ')':
                    break
                ii -= 1
            Term2_l_brct_ind = ii
            #Term2_l_brct_ind = Term1_l_brct_ind-1-item_lgth
            Term2_r_brct_ind = Term1_l_brct_ind-1
            # get what in brackets, without "(" ")"
            store_1 = polynomial_simplify(f_text_v[Term1_l_brct_ind+1:Term1_r_brct_ind]) 
            store_2 = polynomial_simplify(f_text_v[Term2_l_brct_ind:Term2_r_brct_ind])
            # get valute next after bracket # even if it ()
            store_1 = get_monomials(store_1)
            store_2 = get_monomials(store_2)
            mult = [] 
            # monomial multiplication
            for i in range(len(store_1)):
                for j in range(len(store_2)): mult.append(store_1[i] + "*" + store_2[j])
            str_sum = "+".join(mult) 
            # monomial summation
            if(str_new == ""): str_new = str_sum 
            else: str_new = "+".join([str_new, str_sum])
            f_text_v = f_text_v[0:Term2_l_brct_ind] + f_text_v[Term1_r_brct_ind+1:len(f_text_v)]
            # remove used part
            mark = max(mark - Term1_r_brct_ind, 0)
        else: 
            # +()+ case
            store_1 = polynomial_simplify(f_text_v[Term1_l_brct_ind+1:Term1_r_brct_ind]) 
            if(str_new == ""): str_new = store_1 
            else: str_new = "+".join([str_new, store_1])
      
            f_text_v = f_text_v[0:Term1_l_brct_ind] + f_text_v[Term1_r_brct_ind+1:len(f_text_v)]
            mark = max(mark - Term1_r_brct_ind, 0)

    return(str_new)

def note_simplify_id(note, list_l, list_M, dim):
    '''Core function for character multiplication 
    in IRM identification'''
    i = len(note)-1 # № of M in note
    res = 0
  
    while note[i][0] != "M": i = i - 1
    numOfList_1 = int(note[i+1][1:len(note[i+1])]) - 1
    numOfList_2 = int(note[i+2][1:len(note[i+2])]) - 1
    # - 1 needs because lists in note enumeted from 1
    M = list_M[len(list_M)-1]
    #print('M', M)
    list_1 = list_l[numOfList_1]
    if i + 2 <= len(note):
        # there are 2 lists or no Matr
        list_2 = list_l[numOfList_2]
        # get info about vectors in list positions
        list_1_type = isinstance(list_1[1], str)
        list_2_type = isinstance(list_2[1], str)
        # caculate result in different situations
        if not(list_1_type) and not(list_2_type):
            #print('both num   ')
            # both numerical # "types: FALSE FALSE"
            res = selectFromDF(M, list_2, list_1, dim) 
    
        if (list_1_type == False) and (list_2_type == True): 
            # 1 - numerical, 2 - str
            res = selectFromDF(M, [0], list_1, dim) 
            res = Fcomb(list_2, res, dim) 
    
        if list_1_type and list_2_type: 
            #print('both str   ')
            # both str      
            # get M by strings(oh, wide = dim) and throgth Fcomb form one column
            res_comb_t = []
            res = []
            for k in range(dim):
                v = []
                for ii in range(dim):
                    for j in RowDimToRange_2(k,dim):
                        #v = np.hstack(v, M[j,ii])
                        v.append(M[j,ii])
                #print(v)
                res = Fcomb(list_1, v, dim)
                #print('res_l1', res)
                #res_comb_t = np.hstack(res_comb_t, res)
                #res_comb_t.append(res)
                res_comb_t = list(chain(res_comb_t, res))
                # w with lists so have row in output 
            #print("res_comb_t", res_comb_t)
            res = Fcomb(list_2, res_comb_t, dim) 
            #print('res_l2', res)
            # M1*res_2 #vec(2)*vec(4) -> vec(2)

    note[i] = "l{}".format(numOfList_1+1)
    #if i + 3 <= len(note): note = note[0:i+1] + note[i+2:len(note)] # don't see why
    note = note[0:i+1] + note[i+2:len(note)] # del i+1 item
    note = note[0:i+1] + note[i+2:len(note)] # del i+2 item
    return note, res, numOfList_1, numOfList_2

def note_simplify_id_onestep(note, list_1, list_2, M, dim):
    '''Core function for character multiplication 
    in IRM identification. 2 lists mult on M'''
    i = len(note)-1 # № of M in note
    res = 0

    # get info about vectors in list positions
    list_1_type = isinstance(list_1[1], str)
    list_2_type = isinstance(list_2[1], str)
    # caculate result in different situations
    if not(list_1_type) and not(list_2_type): 
        # both numerical # "types: FALSE FALSE"
        res = selectFromDF(M, list_2, list_1, dim) 
    if (list_1_type == False) and (list_2_type == True): 
        res = selectFromDF(M, [0], list_1, dim) 
        res = Fcomb(list_2, res, dim) 
    if list_1_type and list_2_type: 
        # both str      
        # get M by strings(oh, wide = dim) and throgth Fcomb form one column
        res_comb_t = []
        res = []
        for k in range(dim):
            v = []
            for ii in range(dim):
                for j in RowDimToRange_2(k,dim):
                    #v = np.hstack(v, M[j,ii])
                    v.append(M[j,ii])
            res = Fcomb(list_1, v, dim)
            res_comb_t = list(chain(res_comb_t, res))
        res = Fcomb(list_2, res_comb_t, dim) 
        # M1*res_2 #vec(2)*vec(4) -> vec(2)

    return res


def makedimension(name, dim):
    '''create list w dimensions of name'''
    lname = []
    for i in range(dim):
        lname.append(name+'_'+str(i))
    return lname

def select_monem(text, i):
    if text[i].isalpha(): i += 1
    else: return i
    while i < len(text):
        if text[i].isnumeric():
            i += 1
            continue
        else: break
    return i

def select_monem(text, i):
    if text[i].isalpha(): i += 1
    else: return i
    while i < len(text):
        if text[i].isnumeric():
            i += 1
            continue
        else: break
    return i

def traverse_note_ml2sa(container, which_branch):  
    '''convertor ml to shell-alpha (sa)
        return 1 or 2 subtree. 1 - for the right
        container(list) = text, i, branch
       '''
    branch_r = ""
    branch_l = ""
    Matr = ""
    #print(container[0][container[1]])
    
    if container[0][container[1]].isalpha():
        edge = select_monem(container[0], container[1])
        monem = container[0][container[1]:edge]
    else: return container[0][container[1]]
    #print(monem)
    if container[0][container[1]] == 'l':
        #container[2] = monem
        container[2] = digit2alpha(monem[1:len(monem)])
        if(edge < len(container[0])): container[1] = edge
        else: container[1] = len(container[0]) - 1 
        return(container)
    else:
        #if(which_branch == 0): Matr = monem
        if(which_branch == 0): Matr = '('
        if(edge < len(container[0])): container[1] = edge
        else: container[1] = len(container[0]) - 1 
  
    if(which_branch == 0): # std mode
        container = traverse_note_ml2sa(container, 0) 
        branch_r = container[2]
        #print("branch_r", branch_r)
        #container[3].append(branch_r)
        container = traverse_note_ml2sa(container, 0)  # left
        branch_l = container[2]
        #print("branch_l", branch_l)
        #container[3].append(branch_l)
    else:
        container = traverse_note_ml2sa(container, 0) 
        branch_r = container[2]
    
        container = traverse_note_ml2sa(container, 0)  # left
        branch_l = container[2]
        if(which_branch == 1):branch_l = ""
        if(which_branch == 2):branch_r = ""
        which_branch = 0 # do once, for now
  
  
    container[2] = Matr + branch_r + branch_l +')'
    #print(container[[3]])
    return(container)

def traverse_note_get_branches(container, which_branch):  
    '''return 1 or 2 subtree. #211119 changed
        1 - for the right, 2 -for left
        container(list) = text, i, branch'''
    branch_r = ""
    branch_l = ""
    Matr = ""
    #print(container[0][container[1]])
    
    if container[0][container[1]].isalpha():
        edge = select_monem(container[0], container[1])
        monem = container[0][container[1]:edge]
    else: return container[0][container[1]]
    #print(monem)
    if container[0][container[1]] == 'l':
        container[2] = monem
        #container[2] = digit2alpha(monem[1:len(monem)])
        if(edge < len(container[0])): container[1] = edge
        else: container[1] = len(container[0]) - 1 
        return(container)
    else:
        if(which_branch == 0): Matr = monem
        #if(which_branch == 0): Matr = '('
        if(edge < len(container[0])): container[1] = edge
        else: container[1] = len(container[0]) - 1 
  
    if(which_branch == 0): # std mode
        container = traverse_note_get_branches(container, 0) 
        branch_r = container[2]
        #print("branch_r", branch_r)
#         if branch_r[0] == 'M':
#             container[3].append(branch_r)
        
        container = traverse_note_get_branches(container, 0)  # left
        branch_l = container[2]
        #print("branch_l", branch_l)
#         if branch_l[0] == 'M':
#             container[3].append(branch_l)
        container[3].append([Matr, branch_r, branch_l])
    else:
        container = traverse_note_get_branches(container, 0) 
        branch_r = container[2]
    
        container = traverse_note_get_branches(container, 0)  # left
        branch_l = container[2]
        if(which_branch == 1):branch_l = ""
        if(which_branch == 2):branch_r = ""
        which_branch = 0 # do once, for now
  
  
    container[2] = Matr + branch_r + branch_l
    #print(container[[3]])
    return(container)


def traverse_note_get_branches_wm(container, which_branch, m_upper):  
    '''return 1 or 2 subtree. #212104 changed
        1 - for the right, 2 -for left
        container(list) = text, i, branch, output
        output = core node, matr w her branches: l r'''
    branch_r = ""
    branch_l = ""
    Matr = ""
    #print(container[0][container[1]])
    
    if container[0][container[1]].isalpha():
        edge = select_monem(container[0], container[1])
        monem = container[0][container[1]:edge]
    else: return container[0][container[1]]
    #print(monem)
    if container[0][container[1]] == 'l':
        container[2] = monem
        #container[2] = digit2alpha(monem[1:len(monem)])
        if(edge < len(container[0])): container[1] = edge
        else: container[1] = len(container[0]) - 1 
        return(container)
    else:
        if(which_branch == 0): Matr = monem
        #if(which_branch == 0): Matr = '('
        if(edge < len(container[0])): container[1] = edge
        else: container[1] = len(container[0]) - 1 
  
    if(which_branch == 0): # std mode
        container = traverse_note_get_branches_wm(container, 0, Matr) 
        branch_r = container[2]
        
        container = traverse_note_get_branches_wm(container, 0, Matr)  # left
        branch_l = container[2]

        container[3].append([m_upper, Matr, branch_r, branch_l])
  
    container[2] = Matr + branch_r + branch_l
    #print(container[[3]])
    return(container)

def get_leaves_numbers(branch):
    '''return leaves numbers from lm word (grpfrombranch)
    return int'''
    lon = []
    i = 0
    while i < len(branch):
        if branch[i] == 'l': 
            i += 1
            num = ''
            while branch[i].isdigit(): 
                num += branch[i]
                if i < len(branch)-1: i += 1
                else: break
            lon.append(int(num))
        else:
            i += 1
                
    return lon

def get_leaves_numbers_txt(branch):
    '''return leaves numbers from lm word (grpfrombranch)
    return list of str'''
    lon = []
    i = 0
    while i < len(branch):
        if branch[i] == 'l': 
            i += 1
            num = ''
            while branch[i].isdigit(): 
                num += branch[i]
                if i < len(branch)-1: i += 1
                else: break
            lon.append(num)
        else:
            i += 1                
    return lon

def get_leaf_number(branch):
    '''return leaf numbers from lm word (grpfrombranch)
    return int'''
    res = 0
    i = 0
    while i < len(branch):
        if branch[i] == 'l': 
            i += 1
            num = ''
            while branch[i].isdigit(): 
                num += branch[i]
                if i < len(branch)-1: i += 1
                else: break
            res = int(num)
        else:
            i += 1              
    return res

def get_leaves_txt(branch):
    '''return leaves numbers from lm word (grpfrombranch)
    return list of str with l letters'''
    lon = []
    i = 0
    while i < len(branch):
        if branch[i] == 'l': 
            i += 1
            num = 'l'
            while branch[i].isdigit(): 
                num += branch[i]
                if i < len(branch)-1: i += 1
                else: break
            lon.append(num)
        else:
            i += 1
                
    return lon

def get_matrix_txt(branch):
    '''    return M + num'''
    num = ''
    i = 0
    if branch[i] == 'M': 
        num = 'm'
        i += 1
        while branch[i].isdigit(): 
            num += branch[i]
            if i < len(branch)-1: i += 1
            else: break
    else:
        num = 'u'
        i += 1
                
    return num

def get_matrices_txt(text):
    '''return matrices w numbers from lm word (grpfrombranch)
    return list of matr with M letters'''
    lon = []
    i = 0
    while i < len(text):
        if text[i] == 'M': 
            i += 1
            num = 'M'
            while text[i].isdigit(): 
                num += text[i]
                if i < len(text)-1: i += 1
                else: break
            lon.append(num)
        else:
            i += 1
    return lon

def count_notecomplexcity(note):
    '''get list = structure. count number of matrices and 'm' '''
    cnt = 0
    for i in note:
        if i[0] == 'M' or i[0] == 'm': cnt += 1
    return cnt

def makerestriction_pair(lstore): # 211216
    '''input - lists pair. output - product'''
    lres = []
    # умножение каждый с каждым
    for i in range(len(lstore)):
        for j in range(i+1,len(lstore)):
            #if i == j: continue
            lres.append([n+'*'+m for n in lstore[i] for m in lstore[j]])
    return list(chain.from_iterable(lres))

def digit2alpha(txt):
    '''converts string of digits into letters'''
    # add defence from input 0 and bigger than 26
    my_list = txt.split()
    txt_res = ''
#     print(my_list)
#     print(num2alpha[int(my_list[0])])
    for i in range(len(my_list)):
        txt_res += num2alpha[int(my_list[i])] + ' '
    #txt = listToString_s(my_list)
    return txt_res.rstrip()

def checkinpattern_str(txt, pattern):
    # get strings
    # if work w list from begining will be faster
    ltxt = txt.split()
    stxt = set(ltxt)
    return stxt.issubset(set(pattern))

def checkinpattern_list(ltxt, lpattern):
    # get lists
    stxt = set(ltxt)
    return stxt.issubset(set(lpattern))

print(checkinpattern_str('31 22', '1 2 3'))
print(checkinpattern_str('3 2', '1 2 3'))
print(checkinpattern_list([1, 2], [1, 2, 3]))

def style_ml_pair(text):
    # проще через сплит
    txt = 'M'
    i = 0
    while i < len(text):
        if text[i].isdigit() :
            txt += 'l' + text[i]
            i += 1
            continue
        else: 
            i += 1
    return txt

def style_ml_r(note, reminder):
    txt = 'Ml' + listToString(reminder) + note
    return txt

def style_ml(note, reminder):
    txt = 'M' + note + reminder
    return txt

#result = re.search('1 3', '1 2 3')
#if result:
#    print(result[0])

style_ml_pair('1 2')

def getnumoffirstleave(text):
    '''input format like: M1l1l2...'''
    i = 0
    num = 0
    while i < len(text):
        if text[i] == 'l':
            if i+1 <= len(text):
                if text[i+1].isdigit() :
                    if i+2 <= len(text):
                        j = i + 2
                        while text[j].isdigit():
                            j += 1
                            if j == len(text):
                                break
                        num = int(text[i+1:j])
                        break
                    else:
                        num = int(text[i+1])
                        break
        i += 1
    return num

def namem(text):
    txt = ''
    cnt = 1
    for i in range(len(text)):
        if text[i] == 'M':
            txt += 'M' + str(cnt)
            cnt += 1
        else:
            txt += text[i]
    return txt

def get_leaves_numbers_txt_m(branch):
    '''return leaves numbers from lm word (grpfrombranch)
    return str. And for convolved combinations from m_1_2_10_11'''
    lon = []
    i = 0
    while i < len(branch):
        if branch[i] == 'l': 
            i += 1
            num = ''
            while branch[i].isdigit(): 
                num += branch[i]
                if i < len(branch)-1: i += 1
                else: break
            lon.append(num)
        elif branch[i] == 'm' or branch[i] == '_': 
            i += 1
            num = ''
            while branch[i].isdigit(): 
                num += branch[i]
                if i < len(branch)-1: i += 1
                else: break
            lon.append(num)     
        else:
            i += 1              
    return lon

def note_ml_str2vec(note):
    '''convert ml string into ml vector'''
    res = []
    letter = ''
    i = 0
    while i < len(note):
        if note[i].isalpha(): 
            letter = note[i]
            i += 1
            num = ''
            while note[i].isdigit(): 
                num += note[i]
                if i < len(note)-1: i += 1
                else: break
            res.append(letter+num)
        else:
            i += 1
                
    return res

def note_ml_str2vec_(note):
    '''convert ml string into ml vector m with _'''
    res = []
    letter = ''
    i = 0
    while i < len(note):
        if note[i].isalpha(): 
            letter = note[i]
            i += 1
            num = ''
            while note[i].isdigit() or note[i] == "_": 
                num += note[i]
                if i < len(note)-1: i += 1
                else: break
            res.append(letter+num)
        else:
            i += 1
                
    return res

def seekintable(tbl, start, item4seek):
    for i in range(start,-1,-1):
        if tbl[i][0] == item4seek: return i
        
def phrasefromleaves(note, free_term): # note - str
    ''' convert numerical sheme of leaves in vertex into ml notation
    1, 2 3 - goes into M1l1l2l3. 1, 2 not in consideration'''
#    print(" in phrasefromleaves(note, free_term)", note, free_term)
    free_term_pos = 0 # 0 - no use, 1 - right, 2 - left
    rbr = note.split(',')[0]
    lbr = note.split(',')[1]
    rbr_s = rbr.split(' ')
    lbr_s = lbr.split(' ')
    if len(free_term) > 0:
        # test in wich branch free_term is
        #t = listToString_s(get_leaves_numbers_txt_m(free_term))
        t = get_leaves_numbers_txt_m(free_term)
#         print("get_leaves_numbers_txt", t)
#         print("len(t", len(t))
#         print("set(t)", set(t))
#         print("set(rbr)", set(rbr_s))
#         print("set(lbr)", set(lbr_s))
#         print("len(lbr)", len(lbr))
        #t = t.split()
        st = set(t)
 
        #if rbr == t: free_term_pos = 1
        #elif lbr == t: free_term_pos = 2
        if st.issubset(set(rbr_s)): free_term_pos = 1
        elif st.issubset(set(lbr_s)): free_term_pos = 2
        else: print("can't find free_term")
        #print('st.issubset(set(rbr))', st.issubset(set(rbr_s)))
        #print('st.issubset(set(lbr))', st.issubset(set(lbr_s)))
        if free_term_pos == 1: rbr = free_term
        elif free_term_pos == 2: lbr = free_term
    res = ''
    # если пришел свободный член, то просто подставляем его на нужную позицию, без обработки
    if free_term_pos == 0:
        if len(rbr.split()) == 1 and len(lbr.split()) == 2: # 1 and 23
            res = 'Ml' + rbr + 'Ml' + listToString(lbr.split()[0]) + 'l' + listToString(lbr.split()[1])
        if len(rbr.split()) == 2 and len(lbr.split()) == 2: # 23 and 45
            res = 'MMl' + listToString(rbr.split()[0]) + 'l' + listToString(rbr.split()[1]) +\
            'Ml' + listToString(lbr.split()[0]) + 'l' + listToString(lbr.split()[1])
        if len(rbr.split()) == 1 and len(lbr.split()) > 2: # 1 and 234
            res = 'Ml' + rbr + 'm' + listToString__(lbr.split())
        if len(rbr.split()) == 2 and len(lbr.split()) > 2: # 12 and 234
            res = 'MMl' + rbr.split()[0] + 'l' + rbr.split()[1] + 'm' + listToString__(lbr.split())
        if len(rbr.split()) > 2 and len(lbr.split()) > 2: # 123 and 345
            res = 'M' + 'm' + listToString__(rbr.split()) + 'm' + listToString__(lbr.split())
    elif free_term_pos == 1:
        #print("free_term_pos == 1")
        if len(lbr.split()) == 2: # 45
            res = rbr + 'Ml' + lbr.split()[0] + 'l' + lbr.split()[1]
        if len(lbr.split()) > 2: # 234
            res = rbr + 'm' + listToString__(lbr.split())
    elif free_term_pos == 2:
        #print("free_term_pos == 2")
        if len(rbr.split()) == 1: # 1 and 234
            res = 'Ml' + rbr + lbr
            #print("res = 'Ml' + rbr + lbr", res = 'Ml' + rbr + lbr)
        if len(rbr.split()) == 2: # 12 and 234
            res = 'MMl' + rbr.split()[0] + 'l' + rbr.split()[1] + lbr
        if len(rbr.split()) > 2: # 123 and 345
            res = 'm' + listToString__(rbr.split()) + lbr
        #print("res_in", res)
    #note.insert(lvl,res.split(','))
    return res

def mlnotationfromsheme(note): # note - str
    '''rejected vecsion of phrasefromleaves. convert numerical sheme of leaves in vertex into ml notation
    1, 2 3 - goes into M1l1 + m1_2. 1, 2 goes into M1l1l2'''
#    print(" in phrasefromleaves(note, free_term)", note, free_term)
    rbr = note.split(',')[0]
    lbr = note.split(',')[1]
    rbr_s = rbr.split(' ')
    lbr_s = lbr.split(' ')
    res = ''
    if len(rbr.split()) == 1 and len(lbr.split()) == 1: # 1 and 2
        res = 'Ml' + rbr + 'l' + lbr
    if len(rbr.split()) == 1 and len(lbr.split()) >= 2: # 1 and 23,234
        res = 'Ml' + rbr + 'm' + listToString__(lbr.split())
    if len(rbr.split()) >= 2 and len(lbr.split()) >= 2: # 23 and 45, 123 and 345
        res = 'M' + 'm' + listToString__(rbr.split()) + 'm' + listToString__(lbr.split())
    return res

def notefromsheme(sheme, lvl): # lvl - position to change. note - str
    res = ''
    #for i in range(len(sheme)-1,0,-1):
    for i in range(lvl,-1,-1):
        res = phrasefromleaves(sheme[i], res)
    #print("res", res)
        
    #note.insert(lvl,res.split(','))
    return res

def leavenize(note):
    '''get vec and add 'l' in beginig of each pos'''
    for i in range(len(note)):
        note[i] = 'l' + note[i]
    return note

def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)',text) ]
# my_list =['Hello1', 'Hello12', 'Hello29', 'Hello2', 'Hello17', 'Hello25']
# my_list.sort(key=natural_keys)
# print(my_list)

def replacemnum(note, mnum, lnum):
    check = 0
    for i in range(len(note)):
        if note[i][0] == 'm':
            note[i] = 'm' + str(mnum+lnum)
            check = 1
    return check

def mcount(note):
    cnt = 0
    for i in range(len(note)):
        if note[i][0] == 'm':
            cnt += 1
    return cnt

def resetdim(text):
    '''del text after last '_'. Like m_00_1 goes m_00 '''
    res = ''
    i = len(text)-1
    while i > 0:
        if text[i] == '_': break
        i -= 1
    return text[0:i]


False
True
True


In [3]:
#data_ini = pd.read_csv('211208_football_data8.csv', sep=';')
#data_ini = pd.read_csv('211209_id_arctic_l5_data.csv', sep=';')
data_ini = pd.read_excel("220411_arctic_data.xls", sheet_name = "data")
print(data_ini)
#print(data_ini.iloc[:,[1,3]])
#data_ini = pd.read_csv('freeley_tab_d4_cl3_h.csv', sep=';')
#print(data_ini.iloc[:,0:5].duplicated())
lnum = data_ini.shape[1]-1
print("lnum = ", lnum)
data = np.array(data_ini.iloc[:,0:lnum+1])
#print("data: \n" ,data)
dim = 1+data.max()-data.min()
print("dim = ", (data.max()-data.min())+1)
exnum = len(data_ini)

# input at function like ['8', '2 6 7']

    l1  l2  l3  l4  l5  F
0    2   1   1   2   0  2
1    1   0   0   1   0  1
2    1   1   2   0   2  2
3    2   1   2   2   1  2
4    1   1   0   1   1  2
5    1   1   1   1   1  2
6    2   2   2   1   1  2
7    2   2   2   2   2  2
8    1   2   1   2   1  1
9    1   0   1   1   0  1
10   0   1   1   2   1  1
11   2   2   1   2   1  2
12   2   0   1   1   1  2
13   2   2   1   1   0  2
14   2   2   2   2   1  2
15   1   1   2   1   1  2
16   0   1   1   1   1  2
17   0   0   0   1   0  1
18   2   1   1   0   0  1
19   0   1   0   1   0  2
20   2   0   0   2   1  2
21   2   1   1   1   1  2
22   1   1   2   0   1  1
23   2   0   0   0   0  1
24   1   1   0   2   2  2
25   1   0   0   1   1  1
26   1   0   1   2   0  1
27   1   1   1   2   1  1
28   0   0   1   0   0  0
29   1   0   0   0   0  1
30   1   1   1   0   1  1
lnum =  5
dim =  3


In [4]:
# загрузка таблицы разрешенных ветвей grpcontent
#dfgrp = pd.read_excel('220404_psy_grpcontent.xlsx')
#lnum = len(dfgrp[413][0])/2
#dfgrp = pd.read_excel('220523_arct_grpcontent_short.xlsx')
dfgrp = pd.read_excel('220523_arct_grpcontent_short.xlsx')
#dfgrp = pd.read_excel('220523_zenith_grpcontent.xlsx')
print(dfgrp)
lnum = len(dfgrp[5][0])/2

    0    1    2    3    4    5    6    7    8    9   ...     15     16     17  \
0  1 2  1 3  1 4  1 5  2 3  2 4  2 5  3 4  3 5  4 5  ...  1 4 5  2 3 4  2 3 5   
1    1    1    1    1    1    1    1    1    1    1  ...      1      1      1   
2  1 2  1 3  1 4  1 5  2 3  2 4  2 5  3 4  3 5  4 5  ...  5,1 4  4,2 3  5,2 3   
3  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    NaN    NaN    NaN   

      18     19       20       21       22       23         24  
0  2 4 5  3 4 5  1 2 3 4  1 2 4 5  1 3 4 5  2 3 4 5  1 2 3 4 5  
1      1      1        1        1        1        1             
2  5,2 4  5,3 4  4,1 2 3  5,1 2 4  5,1 3 4  5,2 3 4  5,1 2 3 4  
3    NaN    NaN      NaN      NaN      NaN      NaN        NaN  

[4 rows x 25 columns]


In [60]:
# должно происходить движение по таблице  - перебор всех позиций
tbl = dfgrp
trecklist = [[tbl.shape[1]-1, 2, 0]] # structure: #of col in table, # of row in table, # of branch in cell selected

notelist = [[] for i in range(10)] # note of current structure
rmlist = [[[],[]] for i in range(10)] # сохранение результатов для каждой текущей линейно пройденой ячейки - две ветви
rmlist_sum = [[] for i in range(10)] # сохранение результатов для всех ячеек столбца - две ветви
rmlist_res = []
mlist = [[[],[],[]] for i in range(10)] # сохранение результатов m каждой текущей линейно пройденой ячейки
note = [[] for i in range(10)] # запись для решателя
reslist = []
appel_core = []
level = 0 # 
struct_num = 0
iterfromnotsolved = 0 # flag if already did iteration of trecklist[1], when can't solve. Need for evade double iter in c case


print("len(tbl[])", len(tbl[0]))
cnt = 0
while cnt < pow(10,4):
    if not pd.isna(tbl[trecklist[level][0]][trecklist[level][1]]): #and \
        print('enter in the solve block---------------------')
        print('rmlist[level]', rmlist[level])
        print("trecklist, level, cnt", trecklist, level, cnt)
        print("mlist 1", mlist)
        if not rmlist[level][0] and not rmlist[level][1]: # 0 and a cases
            print('a,0')
            print('br', tbl[trecklist[level][0]][trecklist[level][1]].split(','))
            # есть пара ветвей. Передаем их в решатель. Работы со структурой нет, решаем только один уровень
            #gurobi_res = random.randint(1,100) # if ll - trivial
            gurobi_res = onelvlsolver(tbl[trecklist[level][0]][trecklist[level][1]].split(','),
                                      data_ini, mlist[level-1][1], mlist[level-1][2], dim)
            #if gurobi_res > 10: 
            if gurobi_res[0]:
                #print("решили")
                br = tbl[trecklist[level][0]][trecklist[level][1]].split(',')
                if len(br) > 1:
                    mlist[level] = gurobi_res
                    print('br', br)
                    #print('len(br[1].split())', len(br[1].split()))
                    if len(br[0].split()) >= 2 and len(br[1].split()) >= 2: # Mmm 
                        seek = br[trecklist[level][2]] # get a branch
                        #print('seek', seek)
                        res = seekintable(tbl, trecklist[len(trecklist)-1][0], seek)
                        trecklist.append([res, 2, 0])
                        level += 1
                    if len(br[0].split()) == 1 and len(br[1].split()) >= 2: # Mlm 
                        rmlist[level][0].append('Ml'+br[0].split()[0]) # = ['l', Mvalues]
                        trecklist[level][2] = 1
                else: # Mll
                    br = br[0].split()
                    print('br in MLL', br)
                    rmlist[level-1][trecklist[level-1][2]].append('Ml'+br[0]+'l'+br[1])
                    trecklist[level-1][2] = 1
                    del trecklist[level]
                    level -= 1
            else: 
                print("ne решили") # команда на переход к следующему значению в пуле верхнего уровня
                print('rmlist[level-1]', rmlist[level-1])
                print('rmlist[level]', rmlist[level])
                # если не решили на самом верхнем уровне - нужно взять следующую ячейку
                # взять следующую ячейку столбца. Если стб закончился, а решения нет, и если на верхнем
                # уровне была посчитана одна из ветвей, то этот результат нужно обнулить, перед переходом к след ячейке.
                trecklist[level][1] += 1 
                trecklist[level][2] = 0
                rmlist[level][0], rmlist[level][1] = [],[]
                mlist[level] = [[],[],[]]
        elif rmlist[level][0] and not rmlist[level][1]: # b case
            print('b')
            #notelist[level] = br
            note[level] = 'Mrml'
            trecklist[level][2] = 1
            br = tbl[trecklist[level][0]][trecklist[level][1]].split(',')
            seek = br[trecklist[level][2]] # get a branch
            print('seek for br', br)
            #print('seek', seek)
            res = seekintable(tbl, trecklist[len(trecklist)-1][0], seek)
            trecklist.append([res, 2, 0])
            level += 1
        elif rmlist[level][0] and rmlist[level][1]: # c case
            br = tbl[trecklist[level][0]][trecklist[level][1]].split(',')
            # need to add 'M' in word if mergeing 2 complex branches
            if len(br[0].split()) >= 2 and len(br[1].split()) >= 2: # Mmm -> add M in note
                if level > 0:
                    print('c')
                    w = product(rmlist[level][0], rmlist[level][1])
                    for i in w:
                        rmlist[level-1][trecklist[level-1][2]].append('M'+listToString(list(''.join(i))))
                else:
                    print('c2')
                    w = product(rmlist[level][0], rmlist[level][1])
                    for i in w:
                        rmlist_res.append('M'+listToString(list(''.join(i))))
            else:                
                if level > 0:
                    print('c')
                    w = product(rmlist[level][0], rmlist[level][1])
                    for i in w:
                        rmlist[level-1][trecklist[level-1][2]].append(listToString(list(''.join(i))))
                else:
                    print('c2')
                    w = product(rmlist[level][0], rmlist[level][1])
                    for i in w:
                        rmlist_res.append(listToString(list(''.join(i))))
            trecklist[level][2] = 0
            rmlist[level][0], rmlist[level][1] = [],[] # clean res storages
            trecklist[level][1] += 1       
            mlist[level] = [[],[],[]]
            print("mlist c", mlist)
        print('rmlist 2', rmlist)
        #print('rmlist_sum app', rmlist_sum)         
    else:
        print('lvl-- \n')
        del trecklist[level]
        if level > 0: 
            if not rmlist[level-1][0] or not rmlist[level-1][1]: # проверяем были ли решения в рассмотренном столбце, чтобы не вернуться в него через а условие, если решений не было
                trecklist[level-1][1] += 1   # чтобы не заходить повторно в столбец без решений
                trecklist[level-1][2] = 0
                # в этом случае нужно очистить rmlist[level-1], т.к. м.б. что мы смогли найти решения только для одной из ветвей
                rmlist[level-1] = [[],[]]
            notelist[level] = []
            # если поднимаемся, то переписываем rmlist_sum в rmlist[1] верхнего уровня
            #rmlist[level-1][trecklist[level-1][2]].append(rmlist_sum[level]) # сохраняем результат в ячейку той ветви в которую спускались
            rmlist[level] = [[],[]]
            note[level] = []
            mlist[level] = [[],[],[]]
            print("mlist else", mlist)
            level -= 1      
            if level == 0 and trecklist[level][1] >= len(tbl[trecklist[level][0]]): 
                print("trecklist[level][1], len(tbl[trecklist[level][0]])-1", trecklist[level][1], len(tbl[trecklist[level][0]])-1)
                print('\n last string operated \n')
                break
        else: break  
        iterfromnotsolved = 0
    #print("trecklist, level, cnt", trecklist, level, cnt)
    #print("notelist[level]_1", note_ml_str2vec(namem(notefromsheme(notelist, level))))
    #print("struct_num ", struct_num)
    cnt += 1
print("rmlist_res", rmlist_res)       

len(tbl[]) 4
enter in the solve block---------------------
rmlist[level] [[], []]
trecklist, level, cnt [[24, 2, 0]] 0 0
mlist 1 [[[], [], []], [[], [], []], [[], [], []], [[], [], []], [[], [], []], [[], [], []], [[], [], []], [[], [], []], [[], [], []], [[], [], []]]
a,0
br ['5', '1 2 3 4']
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Set parameter PoolSearchMode to value 2
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 42 rows, 496 columns and 250 nonzeros
Model fingerprint: 0xf47fb9ba
Model has 93 general constraints
Variable types: 1 continuous, 495 integer (495 binary)
Coefficient statistics:
  Matrix range     [1e-13, 1e+00]
  Objective range  [1e-34, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve added 238 rows and 0 columns
Presolve removed 0 rows and 33 columns
Presolve time: 0.00s
Presolved: 280 rows

30   1   1   1
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Set parameter PoolSearchMode to value 2
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 22 rows, 181 columns and 104 nonzeros
Model fingerprint: 0x1b448471
Model has 48 general constraints
Variable types: 1 continuous, 180 integer (180 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-34, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve added 114 rows and 0 columns
Presolve removed 0 rows and 16 columns
Presolve time: 0.00s
Presolved: 136 rows, 165 columns, 380 nonzeros
Variable types: 0 continuous, 165 integer (164 binary)
Found heuristic solution: objective 2.0000000

Root relaxation: objective 1.500000e+01, 164 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds    

In [59]:
# проверка одного уровня: Mlm, Mmm
# нужна проверка на примере из эксель
def onelvlsolver(branches_ini, data_ini, dictm_prev, fvalues, dim):
    if len(branches_ini) == 1: # Mll left
        return dim*dim, dictm_prev, fvalues # dim*dim - number of matrix values in result
    res_if_exist = []
    list_M = []
    list_M_vals = [] # M results on each step in sequence form
    list_M_vals_m = [] # M results on each step in matrix form
    #list_mvar = [] # variable results on each step. On first it is the F
    #list_mvar.append(data_ini['F'].to_list())
    currmatrnum = 1

    numofresonstepsaved = 10
    br_right_position = 0
    br_left_position = 1
    resnooptimal = 0
    #no_l_m = 1 # branch data without matrises

    name_m = 'M111'
    # Create matrises and ini of M_values
    list_M.append(m_create_byname(name_m.lower() + '_', dim))
    #print(list_M)

    #solvestep_num = 1 # внизу есть условие проверки для первого цикла == 0
    cnt_iterations = 0
    #for solvestep_num in range(len(solvesteps_s)):
    for solvestep_num in range(0,1):
        resnooptimal = 0
        # предполагаю, что мне приходит список списков. Нужна защита если передадут ['M1', 'M2l1l4', 'M3l2l3']
        dictm = {}
        dictm_oh = {}
        lolm = []
        ldim_rst = []
        dictm_cnt = 0  
        dictm_oh_cnt = 0 
        # теперь заполнить словарь переменными матриц
        rows = list_M[solvestep_num].shape[0]
        cols = list_M[solvestep_num].shape[1]
        for j in range(cols):
            for i in range(rows):
                #print(list_M[solvestep_num][i,j])
                if dictm.get(list_M[solvestep_num][i,j]) is None:
                    dictm.setdefault(list_M[solvestep_num][i,j], dictm_cnt)
                    dictm_cnt += 1
                if dictm_oh.get(resetdim(list_M[solvestep_num][i,j])) is None:
                    dictm_oh.setdefault(resetdim(list_M[solvestep_num][i,j]), dictm_oh_cnt)
                    dictm_oh_cnt += 1
        #print("dictm_oh", dictm_oh)  
        # Относительно каждой матрицы беру ее ветви
        br_right = branches_ini[br_right_position]
        br_left = branches_ini[br_left_position]
        br_right = br_right.split()
        br_left = br_left.split()
        if len(br_right) > 1: br_r_m = 'm1'
        else: br_r_m = 'u'
        if len(br_left) > 1: br_l_m = 'm2'
        else: br_l_m = 'u'
        br_right = ['l'+str(i) for i in br_right]
        br_left = ['l'+str(i) for i in br_left]
        br_left.sort()
        br_right.sort()
        #print('m br', br_r_m, br_l_m)
        #print('br', br_right, br_left)
        
        
        # add solution for Mll case
        

        branches = list(chain(br_right, br_left))
        branches.sort()
        #print('sorted br', branches)
        # create the data table for each step
        if len(branches) == data_ini.shape[1]-1: 
        #if solvestep_num == 0:
            data_step = data_ini
        else:
            # нужна проверка, что во входном словаре есть указанные в ветвях переменные. Иначе все сломается.
            # input like: br ['9', '2 6 7 8'], select them from DT
    #         branches[0] = branches[0].split()
    #         branches[1] = branches[1].split()
    #         branches = list(chain(branches[0], branches[1]))
    #         branches = [int(i) for i in branches]
    #         branches.sort()
    #         branches = ['l'+str(i) for i in branches]
            data_step = data_ini[branches] # формируем таблицу из всех свернутых на пред. шаге листов
            print(data_step)
            data_step = data_step.drop_duplicates()
            data_step.reset_index(inplace = True, drop = True) 
            step_f_num = solvestep_num - 1      
            # нужно заполнить значениями из результата на основе словаря, т.к. таблица изменилась
            #print("branches", branches)
            valuesfromprevstep = []
            for y in range(len(data_step)):
                #leavesfords = get_leaves_txt(listToString(branches_ini))
                leavesfords = data_step.loc[y,branches] # формируем таблицу из всех свернутых на пред. шаге листов
                #print("leavesfords", leavesfords)
                name = listToString(leavesfords.values)
                #print("name", name)
                # create posible names of m variables from previous step
                name1 = 'm1' + '_' + name
                name2 = 'm2' + '_' + name
                #lname1 = makedimension(name1, dim)
                #lname2 = makedimension(name2, dim)
                name = [name1, name2]
                #print("name", name)
                ohvaluefromdict = []
                for u in name:
                    #print('u', u)
                    if not dictm_prev.get(u) is None:
                        #print('try to find dictm_prev.get(lname[u])', dictm_prev.get(u))
                        ohvaluefromdict = fvalues[dictm_prev.get(u)]
                valuesfromprevstep.append(ohvaluefromdict)
            #valuesfromprevstep = list(chain(valuesfromprevstep))
            #print('valuesfromprevstep', valuesfromprevstep)       
            data_step['F'] = valuesfromprevstep
            #print(data_step)
        #print(data_step)
        exnum = len(data_step)

        # the case when need it is 2 leaves and need to fill matrix
        # return vector. no need to make M
#         Mt = np.empty((dim,dim))
#         Mt.fill(np.nan)
#         if len(branches_ini[br_left_position]) <= 1: # Mll case. # заполняем матрицу значениями F и идем на следующую итерацию         
#             adr_col = 0
#             adr_row = 0
#             for i in range(len(data_step)):
#                 adr_col = data_step.loc[i,br_right]
#                 adr_row = data_step.loc[i,br_left]
#                 Mt[adr_row, adr_col] = data_step.loc[i, 'F']
#             #print(Mt)
#             list_M_vals_m.append(Mt)
#             res_if_exist.append(1)
#             continue
        # составление таблицы конфликтов
        dfconfl = pd.DataFrame(columns=[i for i in range(3)],index=range(len(data_step['F'])))
        #dfconfl[2] = pd.Series(data_step['F'])
        dfconfl[2] = data_step['F']

        # нужно создать имена переменных, а заодно и заполнить табл. конфликтов   
        # нужно заполнять таблицу уникальными именами
        for i in range(len(data_step)):
            names = []
            name_r = ''
            name_l = ''
            lname_r = []
            lname_l = []
            if br_r_m[0] != 'u':
                name_r = data_step.loc[i,br_right]
                name_r = listToString(name_r.values)
                name_r = br_r_m + '_' + name_r
                dfconfl.loc[i,0] = name_r
                lname_r = makedimension(name_r, dim)
                #ldim_rst.append(lname_r)
            else:
                dfconfl.iloc[i,0] = data_step.loc[i,br_right]
            if br_l_m[0] != 'u':
                name_l = data_step.loc[i,br_left]
                name_l = listToString(name_l.values)
                name_l = br_l_m + '_' + name_l
                dfconfl.loc[i,1] = name_l
                lname_l = makedimension(name_l, dim)
                #ldim_rst.append(lname_l)
            else:
                dfconfl.iloc[i,1] = data_step.loc[i,br_left]
            #names = list(chain(lname_r, lname_l))
            # fill dictionary and add restrictions into ldim_rst without repeatings
            # also fill oh dictionary for saving oh result
            if len(name_r):
                if dictm_oh.get(name_r) is None:
                        dictm_oh.setdefault(name_r, dictm_oh_cnt)
                        dictm_oh_cnt += 1
            t_4_ldim_rst = []
            for j in lname_r:
                if dictm.get(j) is None:
                    dictm.setdefault(j, dictm_cnt)
                    dictm_cnt += 1
                    t_4_ldim_rst.append(j)
            #print("t_4_ldim_rst", t_4_ldim_rst)
            if len(t_4_ldim_rst): ldim_rst.append(t_4_ldim_rst) # add without repeatings
            if dictm_oh.get(name_l) is None:
                    dictm_oh.setdefault(name_l, dictm_oh_cnt)
                    dictm_oh_cnt += 1
            t_4_ldim_rst = []
            for j in lname_l:
                if dictm.get(j) is None:
                    dictm.setdefault(j, dictm_cnt)
                    dictm_cnt += 1
                    t_4_ldim_rst.append(j)
            #print("t_4_ldim_rst", t_4_ldim_rst)
            if len(t_4_ldim_rst): ldim_rst.append(t_4_ldim_rst) # add without repeatings
        #print("ldim_rst", ldim_rst)
        #print('\n dictm_oh \n', dictm_oh)
        #print('\n dictm \n', dictm)

        # seek for conflicts in dfconfl table        
        restrictions = []
        investigatebothcols = {0}
        #print("isinstance(dfconfl.iloc[0,0], int)", isinstance(dfconfl.iloc[0,0], int))
        #if isinstance(dfconfl.iloc[0,0], int): investigatebothcols = {0}  # only 1 and 0 in first(r) col
        if str(dfconfl.iloc[0,0]).isnumeric(): investigatebothcols = {0}
        else: investigatebothcols = {0,1} # need to seek confl in both columns

        for col in investigatebothcols: # смена ведущего столбца
            #print("col", col)
            col_diversity = set(dfconfl[col])
            for i in col_diversity:
                #print("--i--", i)
                lstore = []
                rstr = []
                dfconfl_s = dfconfl[dfconfl[col] == i]
                #print("dfconfl", dfconfl)
                #print('data_step['F']', data_step['F'])
                #print("dfconfl_s \n", dfconfl_s)
                if len(dfconfl_s) > 1 and len(set(dfconfl_s[2])) > 1: # there are confls
                    for j in range(dim):
                        #print("i, j", i, j)
                        lstore_loc = []
                        col_selected = {0,1}
                        col_selected.discard(col)
                        group = dfconfl_s[dfconfl_s[2] == j][col_selected]
                        #print("  group \n", group)
                        lstore_loc = list(chain.from_iterable(group.values))
                        if len(lstore_loc): lstore.append(lstore_loc)
                    #print("lstore", lstore)

                    if len(lstore):
                        rstr = makerestriction_pair(lstore)
                        #print(rstr)
                        for k in rstr:
                            restrictions.append(k)
        #print('restrictions', restrictions)
        # нужно удалить дублирующиеся ограничения

        restrictions_sep = []
        # не через умножение, а через перечисление как в Clauses
        for i in restrictions:
            pparts = get_productparts(i)
            #print(pparts)
            txt = []
            t = []
            for i, j in zip(makedimension(pparts[0], dim), makedimension(pparts[1], dim)):
                t.append([i, j])
            txt.append(list(chain.from_iterable(t)))
            restrictions_sep.append(list(chain.from_iterable(txt)))
        #print('\n restrictions_sep \n', restrictions_sep)


        # fill ldim_rst for matrices ограничения по размерности
        #for m in range(len(list_M)):
        for j in range(list_M[solvestep_num].shape[1]):
            for i in [i for i in range(list_M[solvestep_num].shape[0]) if not i%dim]:
                tl = []
                for k in range(dim):
                    #print(list_M[solvestep_num][i+k,j])
                    tl.append(list_M[solvestep_num][i+k,j]) 
                ldim_rst.append(tl)
        #print('\n ldim_rst \n', ldim_rst)

        # Составление функционала
        ressum =''
        #i = 1
        for i in range(exnum):
            list_l = []
            name_r = data_step.loc[i,br_right]
            #print('name_r in functional creation', int(name_r))
            if br_r_m[0] == 'u': # just number in right branch, put it into functional (Mlm)
                list_l.append(i2oh(int(name_r), dim))
            else: # Mmm
                name_r = listToString(name_r.values)
                name_r = br_r_m + '_' + name_r
                list_l.append(makedimension(name_r, dim))
            name_l = data_step.loc[i,br_left]
            name_l = listToString(name_l.values)
            name_l = br_l_m + '_' + name_l
            #print("name_r, name_l", name_r, name_l)
            list_l.append(makedimension(name_l, dim))
            #print(list_l)
            #print(list_M[solvestep_num])
            txt = note_simplify_id_onestep(["M1", "l1", "l2"], list_l[0], list_l[1],
                                           list_M[solvestep_num], dim)
            #print(txt)
            # нужно упростить уравнение.
            # take F as F from tbl or as previous step result
    #         if solvestep_num == 0: F = int(data_ini.loc[i,['K']])
    #         else: F = list_mvar[solvestep_num]
            #F = list_mvar[solvestep_num][i]
            F = data_step['F'][i]
            #print('F', F)
            # нужно выбирать компоненту txt[х] в зависимости от значения функции, для данного примера
            res = polynomial_simplify(txt[F])
            #print('res ', i, res)
            ressum += "+" + res

        # составление групп мономов
        Clauses_polymn = get_monomials(ressum)
        #print("\n Clauses_polymn \n", Clauses_polymn)
        # составить из них "and" clauses а потом сумму из этих clauses
        Clauses_polymn_sep = []
        for i in Clauses_polymn:
            pparts = get_productparts(i)
            Clauses_polymn_sep.append(pparts)
        #print("\n Clauses_polymn_sep \n", Clauses_polymn_sep)

        # Create a new model
        model = gp.Model("Irmidcnf")
        # initialize decision variables and objective
        Vars = model.addVars(len(dictm), vtype=GRB.BINARY, name="Vars")
        Cls = model.addVars(len(Clauses_polymn_sep), vtype=GRB.BINARY, name="Cls")
        Obj0 = model.addVar( name="Obj0")
        rst_and = model.addVars(int(len(restrictions_sep)*len(restrictions_sep[0])/2), vtype=GRB.BINARY, name="Rst_and")
        # создаем дополнительные переменные для непоследовательного случая, т.е. умножения 3-х членов
#         if len(Clauses_polymn_sep[0]) > 2:
#             Mul = model.addVars(len(Clauses_polymn_sep), vtype=GRB.BINARY, name="Mul")
        
        # Make restrictions: Link restrictions_sep and literals
#         for i in range(len(restrictions_sep)):
#             objj = gp.quicksum(Vars[dictm[restrictions_sep[i][j]]] * Vars[dictm[restrictions_sep[i][j+1]]] 
#                               for j in range(len(restrictions_sep[i])) if not j%2) # pairs in monome
#             model.addConstr((objj == 0), name = 'CNSTR_Rst'+str(i))
            
        # добавь инициализацию в ограничения
        model.addConstr((Vars[pow(dim,3)] == 1 ), name="CNSTR_varini"+str(0))
        for i in range(1,dim):
            model.addConstr((Vars[pow(dim,3)+i] == 0 ), name="CNSTR_varini"+str(i))

        # Link Xi and notXi from ldim_rst
        for i in range(len(ldim_rst)):
            objj = gp.quicksum(Vars[dictm[ldim_rst[i][j]]] for j in range(len(ldim_rst[i])))
            model.addConstr((objj == 1 ), name="CNSTR_Dim"+str(i))

#         for i in range(len(restrictions_sep)):
#             rst_and= []
#             for j in range(len(restrictions_sep[i])):
#                 if not j%2:
#                     rst_and.append(gp.and_(Vars[dictm[restrictions_sep[i][j]]], Vars[dictm[restrictions_sep[i][j+1]]]))
#             objj = gp.quicksum(rst_and)
#             model.addConstr((objj == 0), name = 'CNSTR_Rst'+str(i))

#        cnt_rst = 0
#         for i in range(len(restrictions_sep)):
#             for j in range(len(restrictions_sep[i])):
#                 if not j%2:
#                     model.addConstr(rst_and[cnt_rst] == gp.and_(Vars[dictm[restrictions_sep[i][j]]], 
#                                                                 Vars[dictm[restrictions_sep[i][j+1]]]),
#                                    name = 'CNSTR_Rst_and'+str(cnt_rst))
#                     cnt_rst += 1
#             objj = gp.quicksum([rst_and[cnt_rst-1],rst_and[cnt_rst]])
#             model.addConstr((objj == 0), name = 'CNSTR_Rst'+str(i))
            
        # Make obj from polynomial: Link Clauses_polymn_sep and literals
#         for i in range(len(Clauses_polymn_sep)):
#             # тут м.б. как 2 так и 3 члена в мономе. Гуроби не поддерживает умножение более чем двух членов
#             if len(Clauses_polymn_sep[0]) > 2:
#                 #mul[i] = math.prod(Vars[dictm[Clauses_polymn_sep[i][0]]], Vars[dictm[Clauses_polymn_sep[i][1]]])
#                 model.addConstr(Mul[i] == math.prod([Vars[dictm[Clauses_polymn_sep[i][0]]], 
#                                                    Vars[dictm[Clauses_polymn_sep[i][1]]]]), name = 'CNSTR_Mul'+str(i))
#                 obj =  math.prod([Mul[i], Vars[dictm[Clauses_polymn_sep[i][2]]]])
#             else:
#                 # поробуй переписать через логическую функцию для общего случая
#                 obj =  math.prod(Vars[dictm[Clauses_polymn_sep[i][j]]] for j in range(len(Clauses_polymn_sep[i])) )
#             model.addConstr(Cls[i] == obj, name = 'CNSTR_Cls'+str(i))
#         model.addConstr(Obj0 == gp.quicksum(Cls), name="CNSTR_Obj0")
        for i in range(len(Clauses_polymn_sep)):
            obj =  gp.and_(Vars[dictm[Clauses_polymn_sep[i][j]]] for j in range(len(Clauses_polymn_sep[i])) )
            model.addConstr(Cls[i] == obj, name = 'CNSTR_Cls'+str(i))

        #model.addConstr(Obj0 == gp.or_(Cls), name="CNSTR_Obj0")
        model.addConstr(Obj0 == gp.quicksum(Cls), name="CNSTR_Obj0")

        blocksymm = gp.quicksum(Vars[j]*pow(2,j) for j in range(len(dictm)-1,-1,-1))
        #blocksymm /= pow(dim,3)*currmatrnum # normalization
        model.update()
        #print("blocksymm", blocksymm)
        a = pow(10,-34) # нужно сделать связь между числом элементов суммы и а, т.к. 2 все-таки в степени
        Obj0 += a*blocksymm # м.б. проблема с Obj0 <= exnum   
        #model.addConstr(Obj0 <= exnum+a*blocksymm, name = 'Obj_max')
        model.addConstr(Obj0 <= exnum+1, name = 'Obj_max')

        # Set optimization objective
        model.setObjective(Obj0, GRB.MAXIMIZE)
        # Save problem
        model.write("irmidcnf.mps")
        model.write("irmidcnf.lp")
        #model.setParam('TimeLimit', 60)
        model.Params.PoolSearchMode = 2
        model.Params.PoolSolutions = numofresonstepsaved
        #model.Params.DualReductions = 0

        #model.feasRelaxS(1,True,True,True)
        # Optimize
        model.optimize()

        status = model.status
        if status == GRB.UNBOUNDED:
            print('The model cannot be solved because it is unbounded')
            
        if status == GRB.OPTIMAL:
            print('The optimal objective is %g' % model.objVal)
            
        if status != GRB.INF_OR_UNBD and status != GRB.INFEASIBLE:
            print('Optimization was stopped with status %d' % status)
            

        # do IIS
        else:
            print('The model is infeasible; computing IIS')
            removed = []

            # Loop until we reduce to a model that can be solved
            while True:

                model.computeIIS()
                print('\nThe following constraint cannot be satisfied:')
                for c in model.getConstrs():
                    if c.IISConstr:
                        print('%s' % c.constrName)
                        # Remove a single constraint from the model
                        removed.append(str(c.constrName))
                        model.remove(c)
                        break
                print('')

                model.optimize()
                status = model.status

                if status == GRB.UNBOUNDED:
                    print('The model cannot be solved because it is unbounded')
                    #sys.exit(0)
                    break
                if status == GRB.OPTIMAL:
                    break
                if status != GRB.INF_OR_UNBD and status != GRB.INFEASIBLE:
                    print('Optimization was stopped with status %d' % status)
                    #sys.exit(0)
                    break

            print('\nThe following constraints were removed to get a feasible LP:')
            print(removed)
            #resnooptimal = 1

        if not resnooptimal:
            objval = model.getAttr(GRB.Attr.ObjVal)
            print("objval, exnum  ", objval, exnum)
            #print('vars in polynome', len(dictm)-pow(dim,3))
            #print('monoms in polynome', len(Clauses_polymn_sep))
            #print("current list_M_vals:")
            #for q in range(currmatrnum):
            #    list_M_vals.append(ohvec2i(model.x[q*pow(dim,3):(q+1)*pow(dim,3)], dim))
            #    #print(np.array(list_split(ohvec2i(model.x[q*pow(dim,3):(q+1)*pow(dim,3)], dim), dim))) # транспонирование не нужно поряжок расположения элементов уже соотв. НА
            #list_res = ohvec2i(model.x[0:len(dictm)], dim)

            #print("model.x[0:len(dictm)]", ohvec2i(model.x[0:len(dictm)], dim), 
            #  "GRB.Attr.PoolObjVal", model.getAttr(GRB.Attr.PoolObjVal))
            #print("final list_M_vals", list_M_vals)
            #print("final list_M_vals_m", list_M_vals_m)
            #print("list_res", list_res)
            #print("list_dictm", list_dictm)
    
    if not resnooptimal:
        return dim*dim, dictm_oh, ohvec2i(model.x[0:len(dictm)], dim) # dim*dim - number of matrix values in result
    else: return 0, 0, 0

    

In [32]:
print("len(reslist)", len(reslist))
pd.DataFrame({'1':reslist[0:len(structs_res_marks)], '2': structs_res_marks })

len(reslist) 0


NameError: name 'structs_res_marks' is not defined

In [26]:
tdf = pd.DataFrame(list_res_oh)
tdf = tdf.T
tdf.to_excel('220329_arctres_pow.xlsx', engine="xlsxwriter", index=False)